In [6]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained('xlm-roberta-base')
len(tok)

250002

In [4]:
tok = AutoTokenizer.from_pretrained('Shitao/RetroMAE_MSMARCO')
len(tok)

30522

In [1]:
import os
import json
chunk_names = [fname for fname in os.listdir('data/augmented/back-translate/') if fname.endswith('.json')]
chunk_names[414]

'500.json'

## pyarrow dataset

In [ ]:
batch = [
    {
        "content": [
            {
                "speaker": 0,
                "utterance": "Hey."
            },
            {
                "speaker": 1,
                "utterance": "Hey, what can I do for you?"
            },
            {
                "speaker": 0,
                "utterance": "I'd like to visit Chicago, Il-Ord, to meet my best friend, so I need to book a plane ticket."
            },
            {
                "speaker": 1,
                "utterance": "Okay."
            },
            {
                "speaker": 0,
                "utterance": "Could you book me a ticket for 09/09?"
            },
            {
                "speaker": 1,
                "utterance": "Please send me the place of origin."
            },
            {
                "speaker": 0,
                "utterance": "My name is Stephanie Carter."
            },
            {
                "speaker": 1,
                "utterance": "May I ask the date of arrival, please?"
            },
            {
                "speaker": 0,
                "utterance": "I'd like to arrive at 09/11."
            },
            {
                "speaker": 1,
                "utterance": "Please send the exact location of your origin."
            },
            {
                "speaker": 0,
                "utterance": "Of course, my origins are Dallas and Fort Worth, TX - DFW."
            },
            {
                "speaker": 1,
                "utterance": "Do you have any other specifications?"
            },
            {
                "speaker": 0,
                "utterance": "Yeah, I need to stop between my journey."
            },
            {
                "speaker": 1,
                "utterance": "I found a Hawaiian airline number 1003, which meets your criteria."
            },
            {
                "speaker": 0,
                "utterance": "All right, please book the tickets."
            },
            {
                "speaker": 1,
                "utterance": "Your ticket is booked."
            },
            {
                "speaker": 0,
                "utterance": "Thank you."
            },
            {
                "speaker": 1,
                "utterance": "Thank you."
            }
        ],
        "source_dataset_name": "AirDialogue",
        "idx_within_source": 187590,
        "idx": 0
    },
    {
        "content": [
            {
                "speaker": 0,
                "utterance": "Hey."
            },
            {
                "speaker": 1,
                "utterance": "Hey, what can I do for you today?"
            },
            {
                "speaker": 0,
                "utterance": "I'm Anthony Martinez from Charlotte, I want to go to Orlando this month to attend the summit, could you book a ticket from the CLT to MCO?"
            },
            {
                "speaker": 1,
                "utterance": "I'm here to help you with that."
            },
            {
                "speaker": 0,
                "utterance": "Thank you, I want to leave for 08/13 and book my return ticket for 08/15, too."
            },
            {
                "speaker": 1,
                "utterance": "Wait a minute."
            },
            {
                "speaker": 0,
                "utterance": "Okay."
            },
            {
                "speaker": 1,
                "utterance": "There's a one-stop flight with a 100 ticket rate, which is provided by Jet Blue airlines."
            },
            {
                "speaker": 0,
                "utterance": "Please continue."
            },
            {
                "speaker": 1,
                "utterance": "I booked your flight 1026."
            },
            {
                "speaker": 0,
                "utterance": "Thank you for your help."
            },
            {
                "speaker": 1,
                "utterance": "Greetings, thank you for choosing our service."
            }
        ],
        "source_dataset_name": "AirDialogue",
        "idx_within_source": 285622,
        "idx": 1
    }
]

In [40]:
names = [
    'MS-DC',
    'MetaLWOZ',
    'MULTIWOZ2_2',
    'SGD',
    'SimJointGEN',
    'KETOD',
    'FRAMES',
    'Disambiguation',
    'ABCD',
    'AirDialogue',
    'BiTOD',
    'Taskmaster1'
]

upper_bound = 96

upper_bounds = {
    'MS-DC': min(upper_bound, 250),
    'MetaLWOZ': min(upper_bound, 100),
    'MULTIWOZ2_2': min(upper_bound, 75),
    'SGD': None,
    'SimJointGEN': upper_bound,
    'KETOD': upper_bound,
    'FRAMES': upper_bound,
    'Disambiguation': min(upper_bound, 60),
    'ABCD': upper_bound,
    'AirDialogue': upper_bound,
    'BiTOD': upper_bound,
    'Taskmaster1': min(upper_bound, 200),
}


from random import shuffle, seed as set_seet
from math import ceil
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from functools import partial
from typing import Tuple, List
from tqdm import tqdm
from mylib.utils.data import Dialogue, ContextResponsePair
import json
import os
import pyarrow as pa
from datasets import load_dataset


def preprocess_dialogue(
        raw_sample,
        tokenizer,
        bound=None,
        user_id=0,
        system_id=1,
    ):
    """convert single dialogue (in DialogStudio format) to list of utterances and list of corresponding speaker ids"""
    
    if is_empty(raw_sample) or is_too_long(raw_sample) or has_only_single_utterance(raw_sample):
        return

    utterances = []
    speakers = []
    
    for turn in raw_sample:
        for sp, item in zip([user_id, system_id], ['user utterance', 'system response']):
            ut = turn[item]
            if ut == '':
                continue
            utterances.append(ut)
            speakers.append(sp)
    
    if bound is not None and any(is_above_bound(ut, tokenizer, bound) for ut in utterances):
        # if there're any utterances with exceeding length
        return
    
    return utterances, speakers


def is_empty(dia):
    return len(dia) == 0


def is_too_long(dia):
    return len(dia) > 10


def has_only_single_utterance(dia):
    return len(dia) == 1 and (dia[0]['user utterance'] == '' or dia[0]['system response'] == '')


def is_above_bound(ut, tokenizer, bound):
    return len(tokenizer(ut)['input_ids']) > bound


def get_record_iterator(name, tokenizer, bound):
    dataset = load_dataset('Salesforce/dialogstudio', name)['train']['log']
    for i, raw_dia in enumerate(dataset):
        dia = preprocess_dialogue(raw_dia, tokenizer, bound)
        if dia is None:
            continue
        utterances, speakers = dia
        dia = Dialogue(
            utterances=utterances,
            speakers=speakers,
            source_dataset_name=name,
            idx_within_source=i,
            # idx=None
        )
        dct = dia.asdict()
        dct['content'] = json.dumps(dct['content'])
        yield pa.RecordBatch.from_pylist([dct])


def train_test_split(data: List[Dialogue], frac=0.9, seed=0):
    """resulting sizes:
    - train: `frac`
    - test: `(1 - frac) // 2`
    - val: `(1 - frac) // 2`"""
    
    set_seet(seed)
    shuffle(data)

    # assign indices after shuffling the dataset
    for i, dia in enumerate(data):
        dia.idx = i

    n_total = len(data)
    train_size = ceil(frac * n_total)
    test_size = (n_total - train_size) // 2
    val_size = n_total - train_size - test_size

    res = {
        'train': data[:train_size],
        'test': data[train_size:train_size+test_size],
        'val': data[train_size+test_size:]
    }

    print('dataset splits sizes:')
    print(f'{n_total=}, {train_size=}, {test_size=}, {val_size=}')

    return res


def make_pairs(dialogues):
    res = []
    for dia in tqdm(dialogues, desc='making pairs'):
        pairs = []
        for i in range(len(dia)-1):
            pairs.append((dia[:i+1], dia[i+1]))
        res.extend(pairs)
    shuffle(res)
    res = [ContextResponsePair(context=c, response=r, idx=i) for i, (c, r) in enumerate(res)]
    return res

In [41]:
from transformers import AutoTokenizer
from collections import defaultdict
from mylib.utils.training import seed_everything
import itertools as it
import pyarrow.dataset as ds

seed_everything(0)

# supress warnings about long sequences
import logging
logging.getLogger("transformers.tokenization_utils_base").setLevel(logging.ERROR)

#! not the same as roberta, replace in future
tokenizer = AutoTokenizer.from_pretrained('microsoft/mpnet-base')

# load datasets from hugging face, parse, filter and merge into single list
record_iterator_list = []
for dataset_name in names:
    iterator = get_record_iterator(dataset_name, tokenizer, upper_bounds[dataset_name])
    record_iterator_list.append(iterator)

chained_iterator = it.chain.from_iterable(record_iterator_list)

In [42]:
itr = iter(chained_iterator)

In [43]:
next(itr)

pyarrow.RecordBatch
content: string
source_dataset_name: string
idx_within_source: int64
----
content: ["[{"utterance": "I'd like 2 tickets to see Zoolander 2 tomorrow at Regal Meridian 16 theater in Seattle at 9:25 PM", "speaker": 0}, {"utterance": "Okay, your purchase of 2 tickets for Zoolander 2 is confirmed.", "speaker": 1}]"]
source_dataset_name: ["MS-DC"]
idx_within_source: [0]

In [5]:
from datasets import load_dataset

dataset = load_dataset('argilla/databricks-dolly-15k-curated-en')
dataset

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 311.64it/s]
Generating train split: 15015 examples [00:00, 183594.95 examples/s]


DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 15015
    })
})

In [6]:
train_test = dataset['train'].train_test_split(test_size=.1, shuffle=True, seed=0)
train_test

DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 13513
    })
    test: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 1502
    })
})

In [7]:
test_val = train_test['test'].train_test_split(test_size=.5, shuffle=True, seed=0)
test_val

DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 751
    })
    test: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 751
    })
})

In [9]:
test_val['val'] = test_val['train']
del test_val['train']
test_val

DatasetDict({
    test: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 751
    })
    val: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 751
    })
})

In [10]:
from datasets import DatasetDict

res_dataset = DatasetDict({
    'train': train_test['train'],
    'test': test_val['test'],
    'val': test_val['val']
})
res_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 13513
    })
    test: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 751
    })
    val: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 751
    })
})

In [14]:
res_dataset.save_to_disk('data-2/testing_hf', num_shards={'train': 10, 'test': 5, 'val': 5})

Saving the dataset (5/5 shards): 100%|██████████| 751/751 [00:00<00:00, 14196.13 examples/s]


In [16]:
from datasets import load_from_disk

dataset = load_from_disk('data-2/testing_hf')
dataset

/home/alekseev_ilya/dialogue-augmentation/VENV/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


DatasetDict({
    train: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 13513
    })
    test: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 751
    })
    val: Dataset({
        features: ['id', 'category', 'original-instruction', 'original-context', 'original-response', 'new-instruction', 'new-context', 'new-response', 'external_id'],
        num_rows: 751
    })
})

In [17]:
dataset['train'][0]

{'id': '12571',
 'category': 'information_extraction',
 'original-instruction': 'Where did the random walk hypothesis come from?',
 'original-context': 'The random walk hypothesis is a financial theory stating that stock market prices evolve according to a random walk (so price changes are random) and thus cannot be predicted.\nThe concept can be traced to French broker Jules Regnault who published a book in 1863, and then to French mathematician Louis Bachelier whose Ph.D. dissertation titled "The Theory of Speculation" (1900) included some remarkable insights and commentary. The same ideas were later developed by MIT Sloan School of Management professor Paul Cootner in his 1964 book The Random Character of Stock Market Prices.[1] The term was popularized by the 1973 book A Random Walk Down Wall Street by Burton Malkiel, a professor of economics at Princeton University,[2] and was used earlier in Eugene Fama\'s 1965 article "Random Walks In Stock Market Prices",[3] which was a less te

In [18]:
dataset['test'][0]

{'id': '12083',
 'category': 'classification',
 'original-instruction': 'Tell me whether these sports require a ball or not to play: basketball, hockey, track and field, soccer, surfing, water polo, karate, football, gymnastics, skateboarding, golf, tennis, boxing, baseball, rugby, weightlifting, softball, rowing',
 'original-context': '',
 'original-response': 'Basketball, soccer, water polo, football, golf, tennis, baseball, rugby, and softball require a ball to play. Hockey, track and field, surfing, karate, gymnastics, skateboarding, boxing, weightlifting, and rowing do not require a ball.',
 'new-instruction': {'user_id': [None],
  'value': ['Tell me whether these sports require a ball or not to play: basketball, hockey, track and field, soccer, surfing, water polo, karate, football, gymnastics, skateboarding, golf, tennis, boxing, baseball, rugby, weightlifting, softball, rowing'],
  'status': ['submitted']},
 'new-context': {'user_id': [None], 'value': [''], 'status': ['submitte

In [19]:
dataset['val'][0]

{'id': '1479',
 'category': 'information_extraction',
 'original-instruction': 'Which version of the church existed in 1809?',
 'original-context': 'The area was settled by Canary Islanders and Acadians from 1779 on and the parish was formed in 1793 with its first church being "little more than a shack". It was replaced by a more permanent one in 1819, which in turn was replaced by this one in 1856, quite close to the site of the 1793 church. The parish\'s governing "Fabrique", or council of wardens, reportedly appointed a committee to measure the Catholic church in Thibodaux, Louisiana, and',
 'original-response': 'The original version of the church still existed in 1809, it was 10 years later that a more permanent one replaced it.',
 'new-instruction': {'user_id': [None],
  'value': ['Which version of the church existed in 1809?'],
  'status': ['submitted']},
 'new-context': {'user_id': [None],
  'value': ['The area was settled by Canary Islanders and Acadians from 1779 on and the pa

In [24]:
from datasets import load_from_disk

dataset = load_from_disk('data-2/train/context-response-pairs/')
dataset

/home/alekseev_ilya/dialogue-augmentation/VENV/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


DatasetDict({
    train: Dataset({
        features: ['content', 'source_dataset_name', 'idx_within_source', 'context', 'response', 'split_index'],
        num_rows: 450750
    })
    test: Dataset({
        features: ['content', 'source_dataset_name', 'idx_within_source', 'context', 'response', 'split_index'],
        num_rows: 25042
    })
    val: Dataset({
        features: ['content', 'source_dataset_name', 'idx_within_source', 'context', 'response', 'split_index'],
        num_rows: 25042
    })
})

In [25]:
dataset['val'][0]

{'content': '[{"utterance": "Hi.", "speaker": 0}, {"utterance": "Hello.", "speaker": 1}, {"utterance": "My name is Timothy Lewis.", "speaker": 0}, {"utterance": "How can I help you?", "speaker": 1}, {"utterance": "I need to book a flight ticket from JFK to PHX. Please help me in booking a flight ticket.", "speaker": 0}, {"utterance": "Can I know your travelling dates?", "speaker": 1}, {"utterance": "My journey dates are from June, 23 to June, 25.", "speaker": 0}, {"utterance": "Do you have any preference?", "speaker": 1}, {"utterance": "I need to depart in the afternoon and I need a connecting flight.", "speaker": 0}, {"utterance": "Sorry, no flights found for your travelling dates.", "speaker": 1}, {"utterance": "No problem, thanks for your information.", "speaker": 0}, {"utterance": "Thank you.", "speaker": 1}]',
 'source_dataset_name': 'AirDialogue',
 'idx_within_source': 217470,
 'context': [[{'speaker': 0,
    'utterance': 'I need to book a flight ticket from JFK to PHX. Please he

In [1]:
from datasets import load_from_disk

dataset = load_from_disk('data-2/source/train')

/home/alekseev_ilya/dialogue-augmentation/VENV/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alekseev_ilya/dialogue-augmentation/VENV/lib/python3.8/site-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)


In [7]:
from mylib.datasets import DynamicContextResponseDataset

dataset = DynamicContextResponseDataset('data-2/source', split='train', context_size=10)
len(dataset)

450750

In [36]:
dataset[0]

{'context': [{'speaker': 0, 'utterance': 'Hi. I am Dorothy Anderson.'},
  {'speaker': 1, 'utterance': 'Hello, how can I assist you today?'},
  {'speaker': 0,
   'utterance': 'I want to cancel my reservation as my lab exam got cancelled. Can you help me with cancellation?'},
  {'speaker': 1, 'utterance': 'Sure, please wait for a moment.'},
  {'speaker': 0, 'utterance': 'Ok.'},
  {'speaker': 1, 'utterance': 'Sorry, no reservation found with your name.'}],
 'target': [{'speaker': 0, 'utterance': "It's Ok. Thanks for your response."},
  {'speaker': 1, 'utterance': 'Thank you for choosing us.'}]}

In [9]:
import string
import random

random.seed(0)

def generate_id(length):
    alphabet = string.ascii_uppercase + string.digits
    return ''.join(random.choice(alphabet) for _ in range(length))

[generate_id(5) for _ in range(10)]

['Y0CQ6',
 '5ZT4W',
 'N6ISI',
 'GQ8JT',
 'GEV49',
 'GW1UN',
 '9427Q',
 'D9AFZ',
 'A5VPU',
 'EMOPJ']